In [2]:
# constants
DATA_PATH = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS = ['DevType']
TECH_COLS = ['LanguageHaveWorkedWith',
             'DatabaseHaveWorkedWith',
             'PlatformHaveWorkedWith',
             'WebframeHaveWorkedWith',
             'MiscTechHaveWorkedWith',
             'ToolsTechHaveWorkedWith',
             'NEWCollabToolsHaveWorkedWith']

MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL =  "data.pkl"
LOG_MODEL_PKL =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [32]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
import seaborn as sns
sns.set()
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression

from matplotlib import pyplot as plt

### _Functions_

In [8]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values=False):
    '''
        Calculates the quality of predictions compared to
        ground truth data using a specified metric function.
        
        Parameters:
        * ground_truth (DataFrame): containing the actual values for each role.
        * predictions (DataFrame): containing the predicted values for each role.
        * metric_function (Function): calculate the quality of predictions against the ground truth.
        * sort_values (Boolean): determines whether the resulting quality scores should be sorted in ascending order.
        
        Returns: the quality scores for each column (role) in the predictions DataFrame.   
    '''
    
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores

## _Initialize_

### _create directories_

In [9]:
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

### _Read data_

In [11]:
data = pd.read_pickle(DATA_PATH) 
data

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
3                       0                        0   
4                       0                        0   
8                       0                        0   
9                       0                        0   
...                   ...                      ...   
83434                   0                        0   
83435                   0                        0   
83436                   0                        0   
83437                   0                        0   
83438                   0                        0   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
3                                                 0                      0   
4                                                 0                      0   
8                                                 0                      0   
9                                                 1                      0   
...                                             ...                    ...   
83434                                             0                      0   
83435                                             0                      0   
83436                                             1                      0   
83437                                             0                      0   
83438                                             0                      0   

                                                                   \
      DevOps specialist Developer, QA or test Developer, back-end   
0                     0                     0                   0   
3                     0                     0                   0   
4                     0                     0                   0   
8                     0                     0                   0   
9                     0                     0                   0   
...                 ...                   ...                 ...   
83434                 0                     0                   1   
83435                 0                     0                   0   
83436                 1                     0                   0   
83437                 0                     0                   1   
83438                 0                     0                   1   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 0   
3                                                 0   
4                                                 1   
8                                                 0   
9                                                 0   
...                                             ...   
83434                                             0   
83435                                             0   
83436                                             0   
83437                                             0   
83438                                             0   

                                                                        ...  \
      Developer, embedded applications or devices Developer, front-end  ...   
0                                               0                    0  ...   
3                                               0                    1  ...   
4                                               0                    0  ...   
8                                               0                    1  ...   
9                                               0                    0  ...   
...                                           ...                  ...  ...   
83434                                           0     

### _Balance classes_

In [18]:
# Check the total samples of roles
roles_df = data["DevType"].copy()
roles_df.sum(axis=0).sort_values()

Developer, game or graphics                        899
Scientist                                         1046
Developer, QA or test                             1135
Database administrator                            1210
Data or business analyst                          1658
Academic researcher                               1708
Engineer, data                                    1941
System administrator                              2069
Developer, embedded applications or devices       2138
Data scientist or machine learning specialist     2460
DevOps specialist                                 3056
Developer, mobile                                 4751
Developer, desktop or enterprise applications     4845
Developer, front-end                              8932
Developer, back-end                              17084
Developer, full-stack                            20655
dtype: int64

In [20]:
# Resample roles
samples_per_class = 1200
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0) 
    
    resampled_roles.append(sub_df)

In [62]:
# Construct dfs
roles_df  = pd.concat(resampled_roles)
data = data.loc[roles_df.index].copy()

In [63]:
roles_df.sum(axis=0).sort_values()

Developer, game or graphics                      1441
Developer, QA or test                            1514
Database administrator                           1765
Developer, embedded applications or devices      1773
Scientist                                        1910
Data or business analyst                         1965
Engineer, data                                   2046
System administrator                             2110
Developer, mobile                                2155
DevOps specialist                                2170
Academic researcher                              2280
Data scientist or machine learning specialist    2576
Developer, front-end                             2614
Developer, desktop or enterprise applications    2690
Developer, full-stack                            5602
Developer, back-end                              5710
dtype: int64

### _Split data_

In [65]:
X_train, X_test, Y_train, Y_test = train_test_split(data.drop("DevType", axis=1), 
                                                    data["DevType"], 
                                                    random_state=0)

C:\Users\DELL\AppData\Local\Temp\ipykernel_3604\493588664.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test, Y_train, Y_test = train_test_split(data.drop("DevType", axis=1),


## _Train models_

### _Initialize MLflow_

In [66]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)